In [1]:
import numpy as np
import pandas as pd 
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import style
style.use('ggplot')
import ML_functions ## a self-created script, known as ML_functions.py
import numpy as np

In [2]:
data = pd.read_csv("forest C vs C.csv")
#data = pd.read_csv('multivariate.csv')
data = data.sort_values(by = "Label",ascending = False)
## Divide data into numerical data and label
status = pd.DataFrame(data["Label"])
data_df = pd.DataFrame(data.iloc[:,1:])
data_df.shape

(388, 337)

In [3]:
data_df

,B9A064,O00187,O00391,O00533,O14786,O14791,O15230,O43866,O75636,O75882,...,Q15910,Q6UX71,Q96EE4,P27487,Q13822,P35030,P04899,Q9UNN8,P30307,Label
235,13.189070,9.029903,9.451866,8.471447,6.299603,10.368394,1.000000,9.539435,10.893973,11.372265,...,1.000000,8.777049,1.0,7.375587,9.090401,10.525970,1.000000,1.0,9.685563,Control
323,1.000000,8.834304,9.256055,8.642149,8.610312,10.242162,1.000000,8.135678,10.989825,11.334537,...,1.000000,8.296410,1.0,6.480773,8.327400,1.000000,1.000000,1.0,1.000000,Control
322,1.000000,8.627372,7.743659,8.845883,1.000000,9.693214,1.000000,8.046227,9.921671,10.495538,...,1.000000,1.000000,1.0,8.038928,1.000000,1.000000,1.000000,1.0,8.078304,Control
102,12.117098,7.333235,8.920492,8.465599,1.000000,10.093014,1.000000,8.922455,10.701251,10.672264,...,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.0,9.734431,Control
321,7.771095,8.868281,9.465477,9.134283,1.000000,11.182106,1.000000,9.489207,10.469340,10.910737,...,1.000000,1.000000,1.0,1.000000,1.000000,10.843160,1.000000,1.0,1.000000,Control
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,9.560794,8.911892,9.231011,9.158430,1.000000,9.871115,1.000000,9.396572,10.883337,10.959091,...,8.414080,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.0,8.890243,Cancer
144,15.300832,8.737837,9.588418,8.436316,8.862250,9.996110,1.000000,8.222691,10.724325,10.979244,...,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,6.446538,1.0,1.000000,Cancer
142,13.784741,9.052897,9.306391,9.364883,1.000000,10.491186,11.001768,9.275203,10.955343,11.144404,...,1.000000,1.000000,1.0,7.494615,1.000000,8.632698,1.000000,1.0,1.000000,Cancer
141,1.000000,8.557125,8.788325,9.353999,1.000000,10.780381,10.166901,7.076804,9.363935,10.930897,...,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,7.050347,1.0,1.000000,Cancer


Remove depletion protein 

In [4]:
data_df = ML_functions.remove_depletion_protein(data_df)

32 columns will be removed


Remove highly correlated proteins 

In [5]:
highcorr  = []
data_df,highcorr = ML_functions.remove_highcorr(data_df)
## Reinsert a protein (complement factor), because P02745,P02746,P02748 
# are found to be important features, but they are all referred to complement factor 
data_df.insert(2,'P02745',data['P02745'])

48 genes are found to be strongly correlated
(388, 261)


Select K best 

In [6]:
X = data_df.iloc[:,:-1]  #independent columns
status   #target column i.e price range
k = ML_functions.univariate_selectKbest(X,status)

Convert Label to numerical

In [7]:
y,labels = ML_functions.convert_label_numerical(data_df)
X = np.asarray(data_df.iloc[:,:-1])
status["Label"]=y

In [8]:
labels

['Cancer', 'Control']

SVM-RFE Feature Ranking 

In [9]:
from sklearn.feature_selection import SelectKBest,f_classif,chi2
X = data_df.iloc[:,1:-1]  #independent columns
y = status   #target column i.e price range
#apply SelectKBest class to extract top 10 best features
bestfeatures = SelectKBest(score_func=chi2, k=10)
fit = bestfeatures.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
print(featureScores.nlargest(20,'Score'))  #print 10 best features

      Specs      Score
85   P07737  71.130625
202  Q9H4G4  68.937021
143  P43121  54.597270
255  P27487  52.942047
230  Q99801  52.424723
34   P01764  38.426600
215  P01609  36.081065
35   P01765  34.245025
118  P18136  33.394268
31   P01717  31.613122
152  P55056  31.183146
258  Q9UNN8  28.053792
247  Q8WZ42  26.221214
59   P04075  24.637111
178  Q15113  23.966834
165  Q03135  23.891124
73   P05154  23.021804
38   P01781  22.605682
153  P55211  21.765368
190  Q7Z7G0  21.670606


Feature_importance selected by RF

In [10]:
index_ = data_df.iloc[:,:-1].columns
feature_importances = ML_functions.RF_feature_selection(X,y,index_)
#for i in feature_importances['importances'][0:15]:
 #   print("{:.3f}".format(i))

C:\Users\457970640\Desktop\Bioinformatics\R forest\real data\ML_functions.py:80: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X,y)


ValueError: arrays must all be same length

### Feature selection (enter below)

In [ ]:
# Anova + RF_immune relevant + complement
selected_features_B = ['P07996', 'P01019', 'P01375','P08185',
                       'P02776','P27169','P02745','P05154', ]

### Split Data and run RF

In [ ]:
from sklearn import model_selection
from sklearn.metrics import precision_score, recall_score,accuracy_score
from sklearn.metrics import roc_curve, auc, confusion_matrix, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
data_RF = data_df[selected_features_B]
#data_RF = data_df.iloc[:,:-1]

## Fit in model
X_train, X_test, y_train, y_test = train_test_split(data_RF, status)
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = y_train.Label.values
y_test = y_test.Label.values
rf = RandomForestClassifier(n_estimators=100, oob_score=True) 


Model training and 10-fold cross validation

In [ ]:
## Cross-Validation
cv_iter = model_selection.StratifiedShuffleSplit(n_splits=10,test_size=1/5)
oob_scores = 0
precision_ = 0 
accuracy_ = 0
recall_ = 0
for train, test in cv_iter.split(X_train, y_train):
    x_train_cv = X_train[train] 
    y_train_cv = y_train[train]
    x_validate_cv = X_train[test] 
    y_validate_cv = y_train[test]
    # Iteratively train the model
    rf.fit(x_train_cv,y_train_cv)
    result = rf.predict(x_validate_cv)
    preci = precision_score (y_validate_cv,result)
    reca = recall_score (y_validate_cv,result)
    accu = accuracy_score (y_validate_cv,result)
    oob_scores += float(rf.oob_score_)
    precision_ += float(preci)
    recall_ += float (reca)
    accuracy_ += float (accu)

precision_RF = "{:.3f}".format(precision_/10)
oobError_RF = "{:.3f}".format(1 - oob_scores/10)
recall_RF = "{:.3f}".format(recall_/10)
accuracy_RF = "{:.3f}".format(accuracy_/10)
error_RF = "{:.3f}".format(1 - accuracy_/10)
print('The RF CV oob error is: ',  oobError_RF)
print('The RF CV precision is: ', precision_RF)
print('The RF CV recall is: ', recall_RF)
print('The RF CV error is: ', error_RF)
print('The RF CV accuracy is: ', accuracy_RF)



Testing phase 

In [ ]:
rf.fit(x_train_cv,y_train_cv)
result = rf.predict(X_test)
preci = precision_score (y_test,result)
reca = recall_score (y_test,result)
accu = "{:.3f}".format(accuracy_score (y_test,result))
print('The RF CV accuracy is: ', accu)
#ML_functions.draw_confusion_matrix(y_test,result)

In [ ]:
ML_functions.draw_confusion_matrix(y_test,result)

In [ ]:
print(X_test.shape)

In [ ]:
# RF testing ROC score 
thres = 0.5
probas_ = rf.predict_proba(X_test) ### predict class probabilities
pred_ = probas_[:, 1] > thres
fpr_RF, tpr_RF, thresholds = roc_curve(y_test, probas_[:, 1])
train_auc_RF = roc_auc_score(y_test,probas_[:,1])
print("ROC-AUC score is: ",train_auc_RF)

ML_functions.draw_ROC(fpr_RF,tpr_RF,train_auc_RF,'RF')

In [ ]:
table_pd = pd.DataFrame({'RF':[precision_RF,oobError_RF,recall_RF,error_RF,accuracy_RF,accu, "{:.3f}".format(train_auc_RF)]},
                        index = ['Precision', 'oobError','Recall','error','Model Accuracy',
                                 'Testing Accuracy','ROC-AUC' ])

### Bootstrapping to calculate confidence interval of AUC and Accuracy

In [ ]:
X_train_bt, X_test_bt, y_train_bt, y_test_bt = train_test_split(data_RF, status)
Train_boot = pd.concat([X_train_bt,y_train_bt],axis=1)
Test_boot = pd.concat([X_test_bt,y_test_bt],axis=1)

In [ ]:
from sklearn.utils import resample
# configure bootstrap
n_iterations = 1000
n_size = int(len(Train_boot) * 2/3)
n_size_test = int(len(Test_boot))

# run boostrap
stats = list()
for i in range(n_iterations):
    train_bootstrap = resample(Train_boot,n_samples = n_size)
    test_bootstrap = resample(Test_boot,n_samples = n_size_test)
    train_value = train_bootstrap.iloc[:,:-1].values
    train_status = train_bootstrap.iloc[:,-1].values
    test_value = test_bootstrap.iloc[:,:-1].values
    test_status = test_bootstrap.iloc[:,-1].values
    #fit model
    rf = RandomForestClassifier(n_estimators=100, oob_score=True) 
    rf.fit(train_value,train_status)
    #evaluate model
    probas_ = rf.predict_proba(test_value) ### predict class probabilities
    fpr_i, tpr_i, thresholds = roc_curve(test_status, probas_[:, 1])
    train_auc = roc_auc_score(test_status,probas_[:,1])
    stats.append(train_auc)

In [ ]:
# confidence intervals 95%
alpha = 0.95
p = ((1.0-alpha)/2.0) * 100
lower = max(0.0, np.percentile(stats, p))
p = (alpha+((1.0-alpha)/2.0)) * 100
upper = min(1.0, np.percentile(stats, p))
print('%.1f confidence interval %.1f%% and %.1f%%' % (alpha*100, lower*100, upper*100))

### SVM model

SVM model fitting and 10-fold cross validation

In [ ]:
# SVM MODEL 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(x_train_cv)
X_validate_scaled = scaler.fit_transform(x_validate_cv)
params_grid = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
               {'kernel': ['linear'], 'C': [1, 10, 100, 1000]},
               {'kernel': ['poly'], 'C': [1, 10, 100, 1000]}
              ]
svm_model = GridSearchCV(SVC(probability = True), params_grid, cv=10)
svm_model.fit(X_train_scaled, y_train_cv)
# View the accuracy score
print('Best SVM accuracy score for training data:', svm_model.best_score_,"\n") 

# View the best parameters for the model found using grid search
print('Best C:',svm_model.best_estimator_.C,"\n") 
print('Best Kernel:',svm_model.best_estimator_.kernel,"\n")
print('Best Gamma:',svm_model.best_estimator_.gamma,"\n")

In [ ]:
final_model = svm_model.best_estimator_
result = final_model.predict(X_validate_scaled)
preci = "{:.3f}".format(precision_score (y_validate_cv,result))
reca = "{:.3f}".format(recall_score (y_validate_cv,result))
SVM_accu_model = "{:.3f}".format(svm_model.best_score_)
erro = "{:.3f}".format(1 - accuracy_score (y_validate_cv,result))
print('The SVM CV precision is: ', preci)
print('The SVM CV recall is: ', reca)
print('The SVM CV error is: ', erro)
print('The SVM CV accuracy is: ', SVM_accu_model)

Testing phase 

In [ ]:
## Accuracy and confusion matrix
X_test_scaled = scaler.transform(X_test)
final_model = svm_model.best_estimator_
result = final_model.predict(X_test_scaled)
SVM_accu_test = "{:.3f}".format(accuracy_score (y_test,result))
print('The testing SVM accuracy is: ',SVM_accu_test )
#ML_functions.draw_confusion_matrix(y_test,result)

## ROC
probas_SVM = final_model.predict_proba(X_test_scaled)
fpr_SVM,tpr_SVM,thresholds = roc_curve(y_test,probas_SVM[:,1])
train_auc_SVM_test = roc_auc_score(y_test,probas_SVM[:,1])
print("train SVM auc score is: ",train_auc_SVM_test)

In [ ]:
ML_functions.draw_confusion_matrix(y_test,result)

### Bootstrapping 

In [ ]:
from sklearn.utils import resample
from sklearn.svm import SVC
# configure bootstrap
n_iterations = 1000
n_size = int(len(Train_boot) * 2/3)
n_size_test = int(len(Test_boot))

# run boostrap
stats_SVM = list()
for i in range(n_iterations):
    train_bootstrap = resample(Train_boot,n_samples = n_size)
    test_bootstrap = resample(Test_boot,n_samples = n_size_test)
    train_value = train_bootstrap.iloc[:,:-1].values
    train_status = train_bootstrap.iloc[:,-1].values
    test_value = test_bootstrap.iloc[:,:-1].values
    test_status = test_bootstrap.iloc[:,-1].values
    #fit model
    scaler = StandardScaler()
    X_train_bootstrap = scaler.fit_transform(train_value)
    X_validate_bootstrap = scaler.fit_transform(test_value)
    svm_model = SVC(kernel = 'rbf',C=1,gamma = 0.0001 ,probability = True)
    svm_model.fit(X_train_bootstrap, train_status) 
    #evaluate model
    probas_SVM = svm_model.predict_proba(X_validate_bootstrap)
    fpr_SVM,tpr_SVM,thresholds = roc_curve(test_status,probas_SVM[:,1])
    train_auc_SVM = roc_auc_score(test_status,probas_SVM[:,1])
    stats_SVM.append(train_auc_SVM)

In [ ]:
# confidence intervals 95%
alpha = 0.95
p = ((1.0-alpha)/2.0) * 100
lower = max(0.0, np.percentile(stats_SVM, p))
p = (alpha+((1.0-alpha)/2.0)) * 100
upper = min(1.0, np.percentile(stats_SVM, p))
print('%.1f confidence interval %.1f%% and %.1f%%' % (alpha*100, lower*100, upper*100))

Model Summary 

In [ ]:
table_pd['SVM'] = [preci,'NA',reca,erro,SVM_accu_model,SVM_accu_test,"{:.3f}".format(train_auc_SVM_test) ]
table_pd

### Draw Performance graph SVM vs RF

In [ ]:
plt.figure()
plt.plot([0, 1], [0, 1], 'k--')
#plt.plot(mean_fpr, mean_tpr_cons, 'b-', label='training cohort (AUC = %.03f)' % train_auc)
plt.plot(fpr_RF, tpr_RF, 'b-', label='RF (AUC = %.03f)' % train_auc_RF)
plt.plot(fpr_SVM, tpr_SVM, 'r-', label='SVM (AUC = %.03f)' % train_auc_SVM_test)
#fpr_i, tpr_i
#plt.plot(fpr_test, tpr_test, 'r-', label='test_cohort (AUC = %.03f)' % test_auc)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('1 - specificity')
plt.ylabel('sensitivity')
plt.title('cancer vs. control')
plt.legend(loc="lower right", prop={'size':10})
plt.show()
plt.close()


### Clustering --- Draw tsne plot

In [ ]:
Perplexity_list = [20,30,50]
x_subset = data_RF.values
y_subset = status.values
label_tsne = list(data.Label)
ML_functions.draw_tsne(x_subset,y_subset,label_tsne,Perplexity_list)

### Clustering --- PCA plot 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import visualisation # self-created script, known as visualisation.py
import plotly
import plotly.express as px
import plotly.graph_objs as go
from sklearn.preprocessing import StandardScaler

##### Change here for feature selection

In [ ]:
RF_feature =  ['P07996', 'P01019', 'P01375','P08185',
                       'P02776','P27169','P02745','P05154', ]
#, 'P19652'
#'Q9H4G4',

#RF_feature = ['P07996', 'P18428', 'P02766', 'P01019', 'P08185',
 #     'P43251', 'P05154', 'P19652','P07737','P27169'] 

In [ ]:
data_RF = data_df[RF_feature]
x = data_RF.iloc[:]
data_ = StandardScaler().fit_transform(x)
data_ = pd.DataFrame(x,columns = RF_feature)
labels = data.iloc[:,-1]
W, scores, fracs = visualisation.do_pca(data_)
scores = scores/abs(scores).max().max() # Scale the scores by the maximum value in the score matrix

In [ ]:
x = np.arange(1,len(fracs)+1)
y = np.cumsum(fracs) # Cumulative sum of elements in the fracs array
fig = go.Figure(data=go.Scatter(x=x, y=y))
fig.update_layout(xaxis_title='Principal component', yaxis_title='Variance explained (cummulative)',
                 xaxis = dict(dtick = 1.0))
fig.show()

### 2D PCA plot 

In [ ]:
XPC = 'PC2' # Principal component shown on x-axis
YPC = 'PC3' # Principal component shown on y-axis
fig = px.scatter(scores, x=XPC, y=YPC, color = labels, hover_data=[XPC,YPC])
fig.update_traces(mode='markers', marker_line_width=1, marker_size=8)
fig.update_layout(title = {'text':'PCA of Binary data',
                          'x':0.5, 'xanchor':'center', 'y':0.98, 'yanchor':'top'},
                  xaxis_title = 'PC2 (Variance = 0.17)',
                  yaxis_title = 'PC3 (Variance = 0.10)',
                  font = dict(size = 24))
fig.show()

In [ ]:
fig = px.scatter_3d(scores, x='PC2', y='PC3', z='PC1', color=labels)
fig.update_layout(title = {'text':'PCA of Binary data',
                          'x':0.5, 'xanchor':'center', 'y':0.98, 'yanchor':'top'},
                  xaxis_title = 'PC2 (Variance = 0.31)',
                  yaxis_title = 'PC3 (Variance = 0.17)',
                  font = dict(size = 14))
fig.show()